<a href="https://colab.research.google.com/github/shahdAls/Ai-week-/blob/main/Copy_of_day1_Pytorch_CNN_MNIST_AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import clear_output


In [2]:
%pip install torch torchvision torchsummary
%pip install numpy
%pip install matplotlib

clear_output()

In [3]:
import torch
import numpy as np
import torch.nn as nn
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data.sampler import SubsetRandomSampler
from torchsummary import summary
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

## Contents

In this notebook, we'll once again attempt to train an Auto Encoder model and generate images from it. The images we'll train on and generate are MNIST images (0-9 digits).
This time, we'll utilize the power of CNN instead of limiting ourselves to linear layers

You need to know:

1. **pytorch** (for impelementation)
2. a bit of **torch dataloaders and datasets** (not necessary but helps understanding how we're loading data)
3. A little bit of **matplotlib** (for result and training trajectory visualization)

In [4]:
batch_size = 64

train_transforms = transforms.Compose([
                        # transforms.RandomCrop(28, padding=4),
                        transforms.ToTensor(),
                    ])

train_data = MNIST(root='./datasets', train=True, download=True, transform=train_transforms)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 58539530.57it/s]


Extracting ./datasets/MNIST/raw/train-images-idx3-ubyte.gz to ./datasets/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 29843728.46it/s]


Extracting ./datasets/MNIST/raw/train-labels-idx1-ubyte.gz to ./datasets/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 24564507.34it/s]


Extracting ./datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to ./datasets/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4824139.98it/s]


Extracting ./datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./datasets/MNIST/raw



In [ ]:
class CNNAutoEncoder(nn.Module):

  def __init__(self, encoding_dim=None):

    super().__init__()


    self.encoder_block = nn.Sequential(
        # input size first time 28
        nn.Conv2d(1, 16, 3, 2, 1),
        # هنا بس اخذت  ابعاد الصورة في الفرست لير
        # 16 filters
        nn.BatchNorm2d(16),
        nn.LeakyReLU(),

        nn.Conv2d(16, 32, 3, 2, 1),
        nn.BatchNorm2d(32),
        nn.LeakyReLU(),
# layner layer does the learning
        nn.Conv2d(32, 64, 3, 2, 1), # kernal size, stried , padding
        nn.BatchNorm2d(64),
        nn.LeakyReLU(),

        nn.Flatten(1, -1), # same to reshape(-1,64,4*4) هنا عملت له فلاتن

        nn.Linear(64*4*4, 128),
        nn.BatchNorm1d(128),
        nn.LeakyReLU(),

        nn.Linear(128, encoding_dim),
        nn.Tanh(),
    )
#
    self.decoder_linear = nn.Sequential(
        nn.Linear(encoding_dim, 128),
        nn.BatchNorm1d(128),
        nn.LeakyReLU(),

        nn.Linear(#TODO, #TODO),
        nn.BatchNorm1d(#TODO),
        nn.LeakyReLU(),
    )

    self.decoder_cnn = nn.Sequential(

        nn.ConvTranspose2d(64, 32, 3,2 ,1),# عكس الماكس بولنق  يكبر
        nn.BatchNorm2d(32),
        nn.LeakyReLU(),

        nn.ConvTranspose2d(32, 16, 3, 2, 1),output_padding=1)# hint: the error you're getting is because of a missing argument here.
                  # لازم اوتبت بادنق هون ولاحيطلع ايرور
        nn.BatchNorm2d(16),
        nn.LeakyReLU(),

        nn.ConvTranspose2d(16, 3, 3, 2, 1, 1),# هنا عدلها من واحد الى 3 لانه من لون قري الى rgb  3
        nn.Sigmoid(),
    )

  def encoder(self, x):
    # Encoder forward pass

    x = self.encoder_block(x)
    return x

  def decoder(self, x):
    # Decoder forward pass

    x = self.decoder_linear(x)
    x = x.reshape((-1, 64, 4, 4))# 2d unfltten convert to 2d ارجعه الى انه يكون
        # عشان ادخله على الدكودر لازم ارجعه 2d بهي الطريقه
    x = self.decoder_cnn(x)

    return x

  def forward(self, x: torch.Tensor):

    single_inp = x.ndim == 3.  # 3 dimensions mean [C, H, W] not [B, C, H, W] so the input is a single image not batch

    # since the layers expect a batch, let's add a 1 dimension to convert it to [1, C, H, W] where 1 becomes batch size
    if single_inp:
      x = x.unsqueeze(dim=0)

    encoder_out = self.encoder(x)
    decoder_out = self.decoder(encoder_out)

    # if the input was single images, the outputs should also be the same way for consistency
    if single_inp:
      encoder_out = encoder_out.squeeze(dim=0)
      decoder_out = decoder_out.squeeze(dim=0)

    return encoder_out, decoder_out

In [ ]:
encoding_dim = #TODO
model = CNNAutoEncoder(encoding_dim=encoding_dim)

In [ ]:
with torch.no_grad():
  summary(model, (1, 28, 28), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 14, 14]             160
       BatchNorm2d-2           [-1, 16, 14, 14]              32
         LeakyReLU-3           [-1, 16, 14, 14]               0
            Conv2d-4             [-1, 32, 7, 7]           4,640
       BatchNorm2d-5             [-1, 32, 7, 7]              64
         LeakyReLU-6             [-1, 32, 7, 7]               0
            Conv2d-7             [-1, 64, 4, 4]          18,496
       BatchNorm2d-8             [-1, 64, 4, 4]             128
         LeakyReLU-9             [-1, 64, 4, 4]               0
          Flatten-10                 [-1, 1024]               0
           Linear-11                  [-1, 128]         131,200
      BatchNorm1d-12                  [-1, 128]             256
        LeakyReLU-13                  [-1, 128]               0
           Linear-14                   

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model.to(device)

num_epochs = #TODO
lr = #TODO
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# the best error mean squre error becuse we compare each pixel>
criterion = #TODO

In [ ]:
train_losses = []

for i in range(num_epochs):
# هنا ما احتاج الy لانه
#unsupervised lersning we donot used labels
  epoch_weighted_loss = 0
  for (X, _) in train_loader:

    X = X.to(device)

    (encoder_outs, decoder_outs) = model(X)

    optimizer.zero_grad()
    loss = criterion(decoder_outs, X)
    loss.backward()
    optimizer.step()

    epoch_weighted_loss += loss.item()*len(X)

  epoch_loss = epoch_weighted_loss/len(train_loader.dataset)

  print(f'epoch {i+1}/{num_epochs}, loss = {epoch_loss}')

  train_losses.append(epoch_loss)

epoch 1/20, loss = 0.11289110354185104
epoch 2/20, loss = 0.04920210972030958
epoch 3/20, loss = 0.03826261648734411


In [ ]:
plt.plot(train_losses)

In [ ]:
# Helper function to display results
def display_image_grid(images, num_rows, num_cols, title_text):

    fig = plt.figure(figsize=(num_cols*3., num_rows*3.), )
    grid = ImageGrid(fig, 111, nrows_ncols=(num_rows, num_cols), axes_pad=0.15)

    for ax, im in zip(grid, images):
        ax.imshow(im, cmap="gray")
        ax.axis("off")

    plt.suptitle(title_text, fontsize=20)
    plt.show()

In [ ]:
# Now we will try some actual generation. We will sample the encodings randomly and then pass them through decoder

rows, cols = 2, 7
sample_encodings = (torch.rand(rows*cols, encoding_dim).to(device) - 0.5) * 2 # encoding space: [-1,1)
with torch.no_grad():
  generations = model.decoder(sample_encodings).cpu()
  generations = generations.reshape(-1, 28, 28, 1)
display_image_grid(generations.squeeze(1), rows, cols, "generated_images")